In [1]:
import gwsurrogate
import numpy as np

import bilby
import gwpy
import WaVel
import WaVel_

import matplotlib.pyplot as plt
%matplotlib inline

# Define constants: speed of light (in m/s), gravitational constant (in m^3/(kg*s^2)), solar mass (in kg) and megaparsec (in m)
c = 2.99e8
G = 6.67e-11
M_sun = 1.99e30
Mpc = 3.09e22

/home/hemantakumar.phurailatpam/anaconda3/envs/bilby/lib/python3.7/site-packages/gwtools/rotations.py:63: UserWarning: Could not import GWFrames, needed for rotations module
  _warnings.warn("Could not import GWFrames, needed for rotations module")
/home/hemantakumar.phurailatpam/anaconda3/envs/bilby/lib/python3.7/site-packages/gwtools/__init__.py:11: UserWarning: Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.
  _warnings.warn("Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.")


setting __package__ to gwsurrogate.new so relative imports work
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new
setting __package__ to gwsurrogate.new so relative imports work
setting __package__ to gwsurrogate.new so relative imports work


In [6]:
import com_fdomain.functions as func

Loaded NRHybSur3dq8 model


In [2]:
outdir = 'outdir'
label = 'moving_bbh'
sampling_frequency = 1024
duration = 4.0
geocent_time = 1126259600.
start_time = geocent_time - duration + 0.02
dt = 1/sampling_frequency
#frequency domain
# Load the surrogate model
surrogate = gwsurrogate.LoadSurrogate('NRHybSur3dq8')

Loaded NRHybSur3dq8 model


In [9]:
# frequency_domain
start_time_gps = geocent_time-4.0+0.02
def moving_bbh(freq_array, total_mass, mass_ratio, luminosity_distance, theta_jn, psi, speed, v_the, v_phi,**kwargs):
    #len(times)=duration*sampling_frequency
    times = time_array_ = np.arange(0.0-duration+0.02,0.02,dt)
    
    arg_ = {'surrogate_':surrogate,'M_':total_mass, 'q_':mass_ratio, 'dis_':luminosity_distance, 'the_':theta_jn, 'phi_':psi, 'v_mag_':speed, 'v_the_':v_the, 'v_phi_':v_phi, 'times_':times, **kwargs}
    fplus = WaVel_.Fn(**arg_)
    h_plus = gwpy.timeseries.TimeSeries(fplus.WaVe()[0], t0=start_time_gps,\
                                        dt=dt, name='Strain', channel=None)
    h_cross = gwpy.timeseries.TimeSeries(fplus.WaVe()[1], t0=start_time_gps,\
                                        dt=dt, name='Strain', channel=None)
    
    fplus = h_plus.fft()
    fcross = h_cross.fft()

    return {'plus': fplus.value, 'cross': fcross.value}

In [7]:
#time domian
# Define the time-domain model
def moving_bbh(times, mass, ratio, distance, theta, phi, speed, v_the, v_phi,**kwargs):
    
    times = times - geocent_time
    arg_ = {'surrogate_':surrogate,'M_':mass, 'q_':ratio, 'dis_':distance, 'the_':theta, 'phi_':phi, 'v_mag_':speed, 'v_the_':v_the, 'v_phi_':v_phi, 'times_':times, **kwargs}
    fplus = WaVel.Fn(**arg_)
    h_plus = fplus.WaVe()[0]
    h_cross = fplus.WaVe()[1]
    
    #print(times[0],times[-1],times[1]-times[0],len(times))
    
    return {'plus': h_plus, 'cross': h_cross}

In [8]:
# cython
import time 

t0 = time.time()
ff = moving_bbh(np.arange(start_time,start_time+duration,dt),650,6,300,45,45,3000,0,0) 

t1 = time.time()
print('Execution took {:.4f}'.format(t1 - t0))

Execution took 1.6375


In [9]:
ff

{'plus': array([-4.82047457e-22, -4.30604367e-22, -3.78980710e-22, ...,
        -8.79197953e-21, -9.60230723e-21, -1.01837855e-20]),
 'cross': array([ 2.89210465e-21,  2.90004858e-21,  2.90721722e-21, ...,
        -7.02387016e-21, -5.65954975e-21, -4.20257039e-21])}

In [5]:
# non cython
import time 

t0 = time.time()
ff = moving_bbh(np.arange(start_time,start_time+duration,dt),650,6,300,45,45,3000,0,0) 

t1 = time.time()
print('Execution took {:.4f}'.format(t1 - t0))

Execution took 1.3219


In [6]:
ff

{'plus': array([-4.99897613e-22, -4.46830924e-22, -3.93570235e-22, ...,
        -1.10938291e-20, -1.07628802e-20, -1.02042008e-20]),
 'cross': array([ 3.00892947e-21,  3.01733545e-21,  3.02492670e-21, ...,
        -4.60214596e-23,  1.44441993e-21,  2.83488247e-21])}